In [52]:
import numpy as np
import time
import os

In [53]:
current_path = os.getcwd()
filename = os.path.join(current_path, "matrix_100x100.csv")
matrix1 = np.genfromtxt(filename, delimiter=',').astype(np.float32)
print(np.shape(matrix1),np.array_equal(matrix1, matrix1.T))

filename = os.path.join(current_path, "matrix_200x200.csv")
matrix2 = np.genfromtxt(filename, delimiter=',').astype(np.float32)
print(np.shape(matrix2),np.array_equal(matrix2, matrix2.T))

filename = os.path.join(current_path, "matrix_400x400.csv")
matrix4 = np.genfromtxt(filename, delimiter=',').astype(np.float32)
print(np.shape(matrix4),np.array_equal(matrix4, matrix4.T))

filename = os.path.join(current_path, "matrix_800x800.csv")
matrix8 = np.genfromtxt(filename, delimiter=',',).astype(np.float32)
print(np.shape(matrix8),np.array_equal(matrix8, matrix8.T))

(100, 100)

 True
(200, 200) True
(400, 400) True
(800, 800) True


In [54]:
EPSILON = 1e-8  # Using exponentiation
MAX_ITERATIONS = 1000

In [55]:
def norm(v):
    return np.sqrt(np.dot(v,v))

def normalize(v):
    return v/norm(v)

In [56]:
def qr_householder(A):
    m, n = A.shape
    R = A.astype(float)  # Avoid unnecessary copy, ensure mutability
    Q = np.eye(m)

    for i in range(n - 1):
        # Compute the Householder vector
        u = R[i:, i].copy()  # Work with a slice; make a copy to modify
        alpha = norm(u)
        u[0] -= alpha  # Create the Householder vector in place
        norm_u = norm(u)

        if norm_u != 0:  # Avoid division by zero
            u /= norm_u

            # Apply the reflection to R in place
            R[i:, i:] -= 2 * np.outer(u, u @ R[i:, i:])
            
            # Apply the reflection to Q in place
            Q[:, i:] -= 2 * np.outer(Q[:, i:] @ u, u)
    return Q, R


In [57]:
def sum_lower_triangle(R, epsilon):
    rows, cols = R.shape
    sum = 0
    for i in range(1, rows):  # Start from row 1 (skip diagonal)
        for j in range(i):  # Only check lower triangle
            sum += abs(R[i, j])
    return sum

In [58]:

def wilkinson_shift(A):
    """Computes the Wilkinson shift for better convergence."""
    d = (A[-2, -2] - A[-1, -1]) / 2
    mu = A[-1, -1] - np.sign(d) * (A[-1, -2] ** 2) / (abs(d) + np.sqrt(d**2 + A[-1, -2]**2))
    return mu

def qr_algorithm_with_shifts(A, max_iterations=1000, epsilon=1e-8):
    """Optimized iterative QR algorithm with shifts to find eigenvalues and eigenvectors."""
    
    Ai = A.copy()
    Q_total = np.eye(A.shape[0])  # Initialize Q accumulation

    for i in range(max_iterations):
        shift = wilkinson_shift(Ai)  # Compute optimal shift
        np.fill_diagonal(Ai, np.diagonal(Ai) - shift)  # Subtract shift from diagonal

        Q, R = qr_householder(Ai)  # QR decomposition
        Ai = R @ Q  # Compute next matrix
        np.fill_diagonal(Ai, np.diagonal(Ai) + shift)  # Add shift back

        Q_total @= Q  # In-place accumulation of Q

        # Check convergence
        convergence = sum_lower_triangle(Ai)

        print(f"Iteration {i}: Convergence = {convergence}")

        if convergence < epsilon:
            print(f"Converged after {i} iterations:\n{Ai}")
            return Ai.diagonal(), Q_total  # Eigenvalues and eigenvectors
    
    raise Exception("Max iterations reached without convergence")

In [59]:
# def is_converged(A, epsilon):
#     """Check if the lower triangular part of A is close to zero."""
#     return np.all(np.abs(np.tril(A, k=-1)) < epsilon)

# def qr_algorithm_recursive(A, max_iterations=1000, epsilon=1e-8, iterations=0, Q_accum=None):
#     if iterations >= max_iterations:
#         raise Exception("Max iterations reached")

#     Q, R = np.linalg.qr(A)  # Compute QR decomposition

#     if Q_accum is None:
#         Q_accum = Q  # Initialize Q accumulation
#     else:
#         Q_accum = Q_accum @ Q  # Accumulate Q for eigenvectors

#     Ai = R @ Q  # Compute next matrix
#     diff = norm(A.diagonal() - Ai.diagonal())

#     #print(f"Iteration {iterations}: diff = {diff}")

#     if is_converged(Ai, epsilon) and diff < epsilon:
#         return Ai.diagonal(), Q_accum  # Eigenvalues and accumulated eigenvectors

#     return qr_algorithm_recursive(Ai, max_iterations, epsilon, iterations+1, Q_accum)

In [60]:
test = np.array([[3,0,1],
                   [0,2,2],
                   [1,2,2]])

In [61]:
def sum_lower_triangle(A):
    """Computes the sum of absolute values in the lower triangle (excluding diagonal)."""
    return np.sum(np.abs(np.tril(A, k=-1)))

def wilkinson_shift(A):
    """Computes the Wilkinson shift for better convergence."""
    d = (A[-2, -2] - A[-1, -1]) / 2
    mu = A[-1, -1] - np.sign(d) * (A[-1, -2] ** 2) / (abs(d) + np.sqrt(d**2 + A[-1, -2]**2))
    return mu

def qr_algorithm_with_shifts_recursive(A, max_iterations=1000, epsilon=1e-8, iterations=0, Q_total=None):
    """Recursive QR algorithm with shifts to find eigenvalues and eigenvectors."""
    
    if iterations >= max_iterations:
        raise Exception("Max iterations reached")

    if Q_total is None:
        Q_total = np.eye(A.shape[0])  # Initialize Q accumulation
    
    shift = wilkinson_shift(A)  # Compute optimal shift
    B = A - np.eye(A.shape[0]) * shift  # Vectorized diagonal subtraction
    
    Q, R = qr_householder(B)  # Compute QR decomposition
    Ai = R @ Q + np.eye(A.shape[0]) * shift  # Compute next matrix (add shift back)
    
    # Accumulate eigenvector transformations
    Q_total = Q_total @ Q

    # Compute convergence using sum of lower triangle elements
    convergence = sum_lower_triangle(Ai)

    print(f"Iteration {iterations}: Convergence = {convergence}")

    if convergence < epsilon:
        print(f"Converged after {iterations} iterations:\n{Ai}")
        return Ai.diagonal(), Q_total  # Return eigenvalues and eigenvectors

    return qr_algorithm_with_shifts_recursive(Ai, max_iterations, epsilon, iterations + 1, Q_total)


In [62]:
matrix = matrix8

In [ ]:
# Measure time for qr_shifts
start_time = time.time()
eigs_qr, eigv_qr = qr_algorithm_with_shifts(test, max_iterations=MAX_ITERATIONS, epsilon = EPSILON)
qr_time = time.time() - start_time
print(f"\nExecution time for qr_shifts: {qr_time:.6f} seconds")

# Measure time for NumPy's eig function
start_time = time.time()
eigs_np, eigv_np = np.linalg.eig(test)
np_time = time.time() - start_time
print(f"Execution time for np.linalg.eig: {np_time:.6f} seconds")

es = np.array(eigs_qr)
ev = np.array(eigv_qr)
print(test @ ev - ev * es)
print()
print(test @ eigv_np - eigv_np * eigs_np)



In [ ]:
# Measure time for qr_shifts
start_time = time.time()
eigs_qr, eigv_qr = qr_algorithm_with_shifts(matrix, max_iterations=MAX_ITERATIONS, epsilon = EPSILON)
qr_time = time.time() - start_time
print(f"\nExecution time for qr_shifts: {qr_time:.6f} seconds")

# Measure time for NumPy's eig function
start_time = time.time()
eigs_np, eigv_np = np.linalg.eig(matrix)
np_time = time.time() - start_time
print(f"Execution time for np.linalg.eig: {np_time:.6f} seconds")

Iteration 0: Convergence = 47377.02775831183
Iteration 1: Convergence = 39821.794733839095
Iteration 2: Convergence = 35593.805122992926
Iteration 3: Convergence = 32487.408600880386
Iteration 4: Convergence = 30021.904610225512
Iteration 5: Convergence = 28008.523005657204
Iteration 6: Convergence = 26313.048585573688
Iteration 7: Convergence = 24868.42945686274
Iteration 8: Convergence = 23617.27675604734
Iteration 9: Convergence = 22521.675760500257
Iteration 10: Convergence = 21551.235929055885
Iteration 11: Convergence = 20691.178054966425
Iteration 12: Convergence = 19922.59764570124
Iteration 13: Convergence = 19228.52431355304
Iteration 14: Convergence = 18589.637309434707
Iteration 15: Convergence = 18001.12818749142
Iteration 16: Convergence = 17461.894931078477
Iteration 17: Convergence = 16966.465421411656
Iteration 18: Convergence = 16507.479228901557
Iteration 19: Convergence = 16078.688348497357
Iteration 20: Convergence = 15678.207785559194
Iteration 21: Convergence = 1

In [47]:
print(f"\nEigenvalues from qr_shifts:\n{eigs_qr}")
print(f"Eigenvalues from np.linalg.eig:\n{eigs_np}")
print(f"\nEigenvectors from qr_shifts:\n{eigv_qr}")
print(f"Eigenvectors from np.linalg.eig:\n{eigv_np}")

print(f"\nEigenvectors difference: {np.linalg.norm(eigv_qr) - np.linalg.norm(eigv_np)}")



Eigenvalues from qr_shifts:
[-2.16424794  2.39138238  0.77286556]
Eigenvalues from np.linalg.eig:
[-2.16424794  0.77286556  2.39138238]

Eigenvectors from qr_shifts:
[[-0.2260912  -0.48280128  0.84604119]
 [-0.6611152  -0.56181831 -0.49727948]
 [ 0.7154086  -0.6717612  -0.19216509]]
Eigenvectors from np.linalg.eig:
[[ 0.2260912  -0.84604119 -0.48280128]
 [ 0.6611152   0.49727948 -0.56181831]
 [-0.7154086   0.19216509 -0.6717612 ]]

Eigenvectors difference: -1.3322676295501878e-15


In [48]:
es = np.array(eigs_qr)
ev = np.array(eigv_qr)
print(matrix @ ev - ev * es)
print()
print(matrix @ eigv_np - eigv_np * eigs_np)

[[ 4.60105826e-09  2.17472351e-09 -1.14667165e-11]
 [ 5.35408962e-09  6.28856145e-09 -1.33432154e-11]
 [ 6.40181597e-09 -6.82234624e-09 -1.59537383e-11]]

[[-2.77555756e-16  0.00000000e+00 -2.22044605e-16]
 [-4.44089210e-16  1.66533454e-16  2.22044605e-16]
 [-6.66133815e-16  5.55111512e-17 -2.22044605e-16]]
